In [9]:
import torch
from torch.utils import data
from torch import nn

In [10]:
def synthetic_data(w, b, num_examples):
    """根据参数生成虚假的数据集"""
    X = torch.normal(0, 1, (num_examples, len(w)))  # 标准正态分布采样 X
    # @ 矩阵乘法计算 如果其中一个参数是向量的话 自动转化为列向量 无需手动转换
    y = X @ w + b  # Y = X*w + b
    y += torch.normal(0, 0.01, y.shape) # 增加高斯噪音
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    # 星号是解包运算符 将 data_arrays 的每个元素分别传递给 TensorDataset
    # 等价于 data.TensorDataset(features, labels)
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[ 1.7263,  0.1552],
         [-2.2767, -2.2426],
         [-0.3445, -0.1878],
         [-1.9460,  1.0994],
         [ 0.3867, -0.9167],
         [-1.3653, -1.2918],
         [-0.8453,  0.0355],
         [-1.5162, -1.5434],
         [-0.5779, -1.2985],
         [ 0.6802, -0.1226]]),
 tensor([[ 7.1175],
         [ 7.2668],
         [ 4.1439],
         [-3.4301],
         [ 8.1004],
         [ 5.8670],
         [ 2.3803],
         [ 6.4390],
         [ 7.4583],
         [ 5.9853]])]

In [12]:
net = nn.Sequential(nn.Linear(2, 1))

net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [13]:
loss = nn.MSELoss()

In [14]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [15]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() # 清空梯度
        l.backward()        # 反向传播计算梯度
        trainer.step()      # 更新参数
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000199
epoch 2, loss 0.000101
epoch 3, loss 0.000100


In [16]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([7.3314e-05, 5.0783e-04])
b的估计误差： tensor([0.0001])
